In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_dataset(dataset):
    encoded = tokenizer(
        dataset["sentence1"],
        dataset["sentence2"],
        padding=True,
        truncation=True,
        return_tensors='np',
    )
    return encoded.data

tokenized_datasets = {
    split: tokenize_dataset(raw_datasets[split]) for split in raw_datasets.keys()
}
train_tokens = tokenized_datasets['train']['input_ids']

In [2]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

checkpoint = 'bert-base-cased'
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
num_train_steps = (len(train_tokens) // batch_size) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
)

In [4]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)
model.compile(loss=loss, optimizer=opt)

ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x000001E73F5737D0>

In [5]:
model.fit(
    tokenized_datasets['train'],
    np.array(raw_datasets['train']['label']), 
    validation_data=(tokenized_datasets['validation'], np.array(raw_datasets['validation']['label'])),
    batch_size=8,
    epochs=3
)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [6]:
preds = model.predict(tokenized_datasets['validation'])['logits']
probabilities = tf.nn.softmax(preds)
class_preds = np.argmax(probabilities, axis=1)

InvalidArgumentError: Graph execution error:

Detected at node tf_bert_for_sequence_classification/bert/embeddings/assert_less/Assert/Assert defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\asyncio\windows_events.py", line 321, in run_forever

  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\asyncio\base_events.py", line 608, in run_forever

  File "c:\Users\usuario\anaconda3\en